In [2]:
# importing the libraries
import graphviz
import snakes.plugins
snakes.plugins.load('gv', 'snakes.nets', 'nets')
from nets import *

In [3]:
# functions for creating the network
def factory (cons, prod, init = [1, 2, 3]):
    n = PetriNet('N')
    n.add_place(Place('src', init))
    n.add_place(Place('tgt', []))
    t = Transition('t')
    n.add_transition(t)
    n.add_input('src', 't', cons)
    n.add_output('tgt', 't', prod)
    return n, t, t.modes()

In [10]:
# Calling the function and firing the transition using Values
net, trans, modes = factory(Value(2), Value(0))
net.draw('value-0.gv')
trans.fire(modes[0])
net.draw('value-1.png')

Graph(['node_2', 'node_1', 'node_0'], [])

In [14]:
# Calling the function and firing the transition using Variables
net, trans, modes = factory(Variable('x'), Variable('x'))
net.draw('value-0.gv')
trans.fire(modes[0])
trans.fire(modes[1])
net.draw('value-1.png')

Graph(['node_2', 'node_1', 'node_0'], [])

In [23]:
net, trans, modes = factory(
    MultiArc([Variable("x"), Variable("y")]),
    MultiArc([Expression("x+y"), Value(0), Expression("x<y")])
)
net.draw("multiarc-0.png")
print(modes)
trans.fire(modes[0])
net.draw("multiarc-1.png")

[Substitution(x=1, y=2), Substitution(x=1, y=3), Substitution(x=2, y=1), Substitution(x=2, y=3), Substitution(x=3, y=1), Substitution(x=3, y=2)]


Graph(['node_2', 'node_1', 'node_0'], [])

In [24]:
net, trans, modes = factory(
    Tuple([Variable("x"), Variable("y")]),
    Tuple([Expression("x+y"), Value(0), Expression("x<y")]),
    [(0,1), (1,2), (2,3)]
)
net.draw("tuple-0.png")
print(modes)
trans.fire(modes[1])
net.draw("tuple-1.png")

[Substitution(x=0, y=1), Substitution(x=1, y=2), Substitution(x=2, y=3)]


Graph(['node_2', 'node_1', 'node_0'], [])

In [3]:
class queue(object):
    def __init__(self, values):
        self.v = list(values)
    def done(self):
        return not self.v
    def next(self):
        return self.v.pop(0)
    def __str__(self):
        return "Queue (%r)" % self.v
    def __repr__(self):
        return "Queue (%r)" % self.v
net = PetriNet('bad')
net.add_place(Place('input'))
net.add_place(Place('output'))
trans = Transition('t', Expression('not x.done()'))
net.add_transition(trans)
net.add_input('input', 't', Variable('x'))
net.add_output('output', 't', Expression('x.next()'))
net.place('input').add(queue(range(10)))


In [4]:
net.get_marking()

Marking({'input': MultiSet([Queue ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])])})

In [5]:
m = trans.modes()
print(m)

[Substitution(x=Queue ([1, 2, 3, 4, 5, 6, 7, 8, 9]))]


In [6]:
trans.fire(m[0])


In [7]:
net.get_marking()

Marking({'output': MultiSet([2])})

In [9]:

class NotBetter (queue) :
    def next (self) :
        print("%s.next()" % self)
        return queue.next(self)

net = PetriNet("not better")
net.add_place(Place("input"))
net.add_place(Place("output"))
trans = Transition("t", Expression("not x.done()"))
net.add_transition(trans)
net.add_input("input", "t", Variable("x"))
net.add_output("output", "t", Expression("x.next()"))
net.place("input").add(NotBetter(range(10)))
print("calling trans.modes()")
m = trans.modes()
print("calling trans.fire()")
trans.fire(m[0])

calling trans.modes()
Queue ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).next()
calling trans.fire()
Queue ([1, 2, 3, 4, 5, 6, 7, 8, 9]).next()
Queue ([2, 3, 4, 5, 6, 7, 8, 9]).next()
